In [2]:
import matplotlib.pyplot as plt
from matplotlib import gridspec
import seaborn as sns
import pandas as pd
from tqdm import tqdm
import numpy as np
from scipy.special import expit
import numpy as np

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, KFold, StratifiedKFold, GridSearchCV
from sklearn.metrics import classification_report
from sklearn.model_selection import ParameterGrid
from sklearn.model_selection import LeaveOneOut, KFold
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [2]:
np.random.seed(20)

In [3]:
df = pd.read_csv("./wisc_bc_data.csv")
len(df)

569

In [4]:
df.head(5)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [5]:
df.diagnosis.unique()

array(['M', 'B'], dtype=object)

In [6]:
df.diagnosis = df.diagnosis.astype(str)
df.diagnosis =  df.diagnosis.apply(lambda x: 1 if  x == "M" else -1)

In [7]:
df.head(2)

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,1,17.99,10.38,122.8,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,1,20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902


In [8]:
df.groupby(["diagnosis"]).describe()

id                                                          \
           count          mean           std     min       25%       50%   
diagnosis                                                                  
-1         357.0  2.654382e+07  1.167397e+08  8913.0  874662.0  908916.0   
 1         212.0  3.681805e+07  1.378965e+08  8670.0  861345.0  895366.5   

                                   radius_mean             ... symmetry_worst  \
                  75%          max       count       mean  ...            75%   
diagnosis                                                  ...                  
-1         8812816.00  911320502.0       357.0  12.146524  ...       0.298300   
 1         8911289.75  911296202.0       212.0  17.462830  ...       0.359225   

                  fractal_dimension_worst                               \
              max                   count      mean       std      min   
diagnosis                                                                
-1         0.4228                   357.0  0.079442  0.013804  0.05521   
 1         0.6638                   212.0  0.091530  0.021553  0.05504   

                                                
                25%      50%       75%     max  
diagnosis                                       
-1         0.070090  0.07712  0.085410  0.1486  
 1         0.076302  0.08760  0.102625  0.2075  

[2 rows x 248 columns]

In [9]:
ratio_classes = 212.0/569
ratio_classes # примерно  37 процентов класс M, классы несбалансированы

0.37258347978910367

In [10]:
y = df.diagnosis.values
X = df.iloc[:,2:].values
scaler = StandardScaler() # засклейим
X = scaler.fit_transform(X)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

In [12]:
y_train[y_train==1].sum()/ len(y_train)# пропорции норм

0.37362637362637363

In [13]:
log_reg = LogisticRegression(penalty='l1', solver='liblinear', C=1.0)
log_reg.fit(X_train, y_train)
print("Коэффициенты:", log_reg.coef_)
selected_features = np.where(log_reg.coef_[0] != 0)[0]
print("Выбранные признаки:", selected_features)
X_train_selected = X_train[:, selected_features]
X_test_selected = X_test[:, selected_features]
model_final = LogisticRegression()
model_final.fit(X_train_selected, y_train)

# Оценка точности модели на тестовой выборке

y_pred = model_final.predict(X_test_selected)
report = classification_report(y_test, y_pred)
print(report)

Коэффициенты: [[ 0.          0.          0.          0.          0.          0.
   0.          1.94831487  0.          0.          2.59761674 -0.69674345
   0.          0.          0.35824865 -1.00439707  0.          0.11784933
   0.         -0.25879246  0.09416868  2.09380004  0.          3.75679638
   0.22382048  0.          1.03562348  0.40022704  0.65826178  0.        ]]
Выбранные признаки: [ 7 10 11 14 15 17 19 20 21 23 24 26 27 28]
              precision    recall  f1-score   support

          -1       0.97      1.00      0.99        72
           1       1.00      0.95      0.98        42

    accuracy                           0.98       114
   macro avg       0.99      0.98      0.98       114
weighted avg       0.98      0.98      0.98       114



In [19]:
def  solver(random_seed, test_size, X, y):
    np.random.seed(random_seed)
    X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=test_size, stratify=y)

    cross_val = StratifiedKFold(5, shuffle = True)
    number_of_batch = cross_val.get_n_splits(X_train)
    gammas = [1e-10, 1e-4, 1e-3, 1e-2, .1, 1., 10., 1e2, 1e3, 1e4, 1e10]
    gamma_scores  = dict()
    for  gamma in gammas:
        list_of_scores = []
        for train_index, test_index in tqdm(cross_val.split(X_train, y_train), 
                                        total=number_of_batch, leave=False):
            x_train_fold = X_train[train_index]
            x_test_fold = X_train[test_index]
            y_train_fold = y_train[train_index]
            y_test_fold = y_train[test_index]
            
            model = LogisticRegression(penalty='l2', C=2/gamma, solver='liblinear',fit_intercept=True, max_iter=2000)
            model.fit(x_train_fold, y_train_fold)
            y_pred_fold = model.predict(x_test_fold)
            list_of_scores.append(f1_score(y_test_fold, y_pred_fold, average='weighted'))
            
        gamma_scores[gamma] = np.mean(list_of_scores)
        
    best_gamma = sorted(gamma_scores, 
                        key=lambda x: gamma_scores[x], reverse=True)[0]
    
    model = LogisticRegression(penalty='l2', C=2/best_gamma, fit_intercept=True,  solver='liblinear', max_iter=2000)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    print("LOG REG + L2 solver")
    print(f'Лучшие параметры:, gamma:{best_gamma}')
    report = classification_report(y_test, y_pred)
    print(report)
    #########################
    param_grid = {
        'C': [0.001, 0.1, 1, 10, 100],
        'kernel': ['linear', 'rbf', 'poly'],
        'gamma': ['scale', 'auto', 0.001, 0.01, 0.1, 1, 10],
        'degree' : [2,3,4,5]
    }
    
    model = SVC()
    stratified_kfold = StratifiedKFold(n_splits=5)
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=stratified_kfold, scoring='f1_weighted', n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("SVM solver")
    print("Лучшие параметры:", best_params)
    best_model = grid_search.best_estimator_
    
    y_pred = best_model.predict(X_test)
    print(classification_report(y_test, y_pred))
    ############################
    param_grid = {
        'criterion': ['gini', 'entropy'],  # Критерий разделения
        'max_depth': [None, 10, 20, 30, 40],  # Максимальная глубина дерева
        'min_samples_split': [2, 5, 10],  # Минимальное количество образцов для разделения
        'min_samples_leaf': [1, 2, 4],  # Минимальное количество образцов в листовом узле
    }
    model = DecisionTreeClassifier()
    grid_search = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv=5, n_jobs=-1, verbose=1)
    grid_search.fit(X_train, y_train)
    best_params = grid_search.best_params_
    print("Decision Trees solver")
    print("Лучшие параметры:", best_params)
    best_model = grid_search.best_estimator_
    y_pred = best_model.predict(X_test)
    
    # Вывод отчета о классификации
    print(classification_report(y_test, y_pred))

In [20]:
solver(300, 0.28, X,y)

LOG REG + L2 solver
Лучшие параметры:, gamma:1.0
              precision    recall  f1-score   support

          -1       0.95      0.99      0.97       100
           1       0.98      0.92      0.95        60

    accuracy                           0.96       160
   macro avg       0.97      0.95      0.96       160
weighted avg       0.96      0.96      0.96       160

Fitting 5 folds for each of 420 candidates, totalling 2100 fits


C:\Users\kuzin\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\ma\core.py:2820: RuntimeWarning: invalid value encountered in cast
  _data = np.array(data, dtype=dtype, copy=copy,


SVM solver
Лучшие параметры: {'C': 0.1, 'degree': 2, 'gamma': 'scale', 'kernel': 'linear'}
              precision    recall  f1-score   support

          -1       0.95      0.99      0.97       100
           1       0.98      0.92      0.95        60

    accuracy                           0.96       160
   macro avg       0.97      0.95      0.96       160
weighted avg       0.96      0.96      0.96       160

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Decision Trees solver
Лучшие параметры: {'criterion': 'entropy', 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5}
              precision    recall  f1-score   support

          -1       0.94      0.93      0.93       100
           1       0.89      0.90      0.89        60

    accuracy                           0.92       160
   macro avg       0.91      0.92      0.91       160
weighted avg       0.92      0.92      0.92       160



In [21]:
solver(13, 0.24, X,y)

LOG REG + L2 solver
Лучшие параметры:, gamma:1.0
              precision    recall  f1-score   support

          -1       0.96      1.00      0.98        86
           1       1.00      0.92      0.96        51

    accuracy                           0.97       137
   macro avg       0.98      0.96      0.97       137
weighted avg       0.97      0.97      0.97       137

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
SVM solver
Лучшие параметры: {'C': 10, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
              precision    recall  f1-score   support

          -1       0.98      1.00      0.99        86
           1       1.00      0.96      0.98        51

    accuracy                           0.99       137
   macro avg       0.99      0.98      0.98       137
weighted avg       0.99      0.99      0.99       137

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Decision Trees solver
Лучшие параметры: {'criterion': 'entropy', 'max_depth': 40, 'min_

In [22]:
solver(8, 0.2, X,y)

LOG REG + L2 solver
Лучшие параметры:, gamma:1.0
              precision    recall  f1-score   support

          -1       0.96      0.99      0.97        72
           1       0.97      0.93      0.95        42

    accuracy                           0.96       114
   macro avg       0.97      0.96      0.96       114
weighted avg       0.97      0.96      0.96       114

Fitting 5 folds for each of 420 candidates, totalling 2100 fits
SVM solver
Лучшие параметры: {'C': 10, 'degree': 2, 'gamma': 0.01, 'kernel': 'rbf'}
              precision    recall  f1-score   support

          -1       0.97      0.99      0.98        72
           1       0.98      0.95      0.96        42

    accuracy                           0.97       114
   macro avg       0.97      0.97      0.97       114
weighted avg       0.97      0.97      0.97       114

Fitting 5 folds for each of 90 candidates, totalling 450 fits
Decision Trees solver
Лучшие параметры: {'criterion': 'entropy', 'max_depth': None, 'mi